## Instructions

You are asked to complete the following files:
* **pruned_layers.py**, which contains the pruning of DNNs to reduce the storage of insignificant weight parameters with 2 methods: pruning by percentage and prune by standara deviation.
* **train_util.py**, which includes the training process of DNNs with pruned connections.
* **quantize.py**, which applies the quantization (weight sharing) part on the DNN to reduce the storage of weight parameters.
* **huffman_coding.py**, which applies the Huffman coding onto the weight of DNNs to further compress the weight size.

You are asked to submit the following files:
* **net_before_pruning.pt**, which is the weight parameters before applying pruning on DNN weight parameters.
* **net_after_pruning.pt**, which is the weight paramters after applying pruning on DNN weight parameters.
* **net_after_quantization.pt**, which is the weight parameters after applying quantization (weight sharing) on DNN weight parameters.
* **codebook_vgg16.npy**, which is the quantization codebook of each layer after applying quantization (weight sharing).
* **huffman_encoding.npy**, which is the encoding map of each item within the quantization codebook in the whole DNN architecture.
* **huffman_freq.npy**, which is the frequency map of each item within the quantization codebook in the whole DNN. 

To ensure fair grading policy, we fix the choice of model to VGG16_half, which is a down-scaled version of VGG16 using a width multiplier of 0.5. You may check the implementation in **vgg16.py** for more details.

In [ ]:
from vgg16 import VGG16, VGG16_half
from train_util import train, finetune_after_prune, test
from quantize import quantize_whole_model
from huffman_coding import huffman_coding
from summary import summary
import torch
import numpy as np
from prune import prune

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

### Full-precision model training

In [8]:
net = VGG16_half()
net = net.to(device)

# Uncomment to load pretrained weights
net.load_state_dict(torch.load("./models/net_before_pruning.pt"))

# Comment if you have loaded pretrained weights
# Tune the hyperparameters here.
# train(net, epochs=75, batch_size=256, lr=0.01, reg=5e-3)

RuntimeError: CUDA error: no CUDA-capable device is detected

In [3]:
# Load the best weight paramters
# net.load_state_dict(torch.load("./models/net_before_pruning.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3012, Test accuracy=0.9171


In [4]:
print("-----Summary before pruning-----")
summary(net)
print("-------------------------------")

-----Summary before pruning-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		864			0.000000
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		9216			0.000000
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		18432			0.000000
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		36864			0.000000
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		73728			0.000000
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		147456			0.000000
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		147456			0.000000
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		294912			0.000000
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		589824			0.000000
26		BatchNorm	N/A		N/A			N/A
27		ReLU		N/A		N/A			N/A
28		Convolutional	589824		589824			0.000000
29		Batch

### Pruning & Finetune with pruned connections

In [5]:
# Test accuracy before fine-tuning
prune(net, method='std', q=45.0, s=0.75)#0.25
test(net)

Files already downloaded and verified
Test Loss=0.5017, Test accuracy=0.8575


In [6]:
# Uncomment to load pretrained weights
# net.load_state_dict(torch.load("net_after_pruning.pt"))
# Comment if you have loaded pretrained weights
finetune_after_prune(net, epochs=50, batch_size=128, lr=0.001, reg=5e-5)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=16]	Loss=0.1665	acc=0.9595	1336.9 examples/second
[Step=32]	Loss=0.1634	acc=0.9590	3238.5 examples/second
[Step=48]	Loss=0.1693	acc=0.9575	3379.2 examples/second
[Step=64]	Loss=0.1707	acc=0.9562	3392.7 examples/second
[Step=80]	Loss=0.1683	acc=0.9560	3191.0 examples/second
[Step=96]	Loss=0.1724	acc=0.9541	3314.2 examples/second
[Step=112]	Loss=0.1689	acc=0.9557	3335.9 examples/second
[Step=128]	Loss=0.1681	acc=0.9552	3235.9 examples/second
[Step=144]	Loss=0.1670	acc=0.9559	3095.9 examples/second
[Step=160]	Loss=0.1642	acc=0.9566	3159.6 examples/second
[Step=176]	Loss=0.1638	acc=0.9569	3179.2 examples/second
[Step=192]	Loss=0.1643	acc=0.9565	2916.7 examples/second
[Step=208]	Loss=0.1637	acc=0.9567	2957.4 examples/second
[Step=224]	Loss=0.1642	acc=0.9565	2872.3 examples/second
[Step=240]	Loss=0.1647	acc=0.9564	2866.5 examples/second
[Step=256]	Loss=0.1637	acc=0.9566	2783.6 exa

In [11]:
# Load the best weight paramters
net.load_state_dict(torch.load("./models/net_after_pruning_after_finetune.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3435, Test accuracy=0.9114


In [8]:
print("-----Summary After pruning-----")
summary(net)
print("-------------------------------")

-----Summary After pruning-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		274			0.682870
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		2931			0.681966
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		7667			0.584039
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		16324			0.557183
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		32708			0.556369
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		66450			0.549357
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		65226			0.557658
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		127939			0.566179
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		248262			0.579091
26		BatchNorm	N/A		N/A			N/A
27		ReLU		N/A		N/A			N/A
28		Convolutional	589824		236636			0.598802
29		BatchNorm

### Quantization

In [12]:
centers = quantize_whole_model(net, bits=5)
np.save("codebook_vgg16.npy", centers)

Complete 1 layers quantization...
Complete 2 layers quantization...
Complete 3 layers quantization...
Complete 4 layers quantization...
Complete 5 layers quantization...
Complete 6 layers quantization...
Complete 7 layers quantization...
Complete 8 layers quantization...
Complete 9 layers quantization...
Complete 10 layers quantization...
Complete 11 layers quantization...
Complete 12 layers quantization...
Complete 13 layers quantization...
Complete 14 layers quantization...
Complete 15 layers quantization...
Complete 16 layers quantization...


In [13]:
net = net.to(device)

In [14]:
test(net)

Files already downloaded and verified
Test Loss=0.3512, Test accuracy=0.9063


In [12]:
print("-----Summary After Quantization-----")
summary(net)
print("-------------------------------")

-----Summary After Quantization-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		864			0.000000
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		9216			0.000000
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		18432			0.000000
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		36864			0.000000
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		73728			0.000000
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		147456			0.000000
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		147456			0.000000
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		294912			0.000000
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		589824			0.000000
26		BatchNorm	N/A		N/A			N/A
27		ReLU		N/A		N/A			N/A
28		Convolutional	589824		589824			0.000000
29		B

### Huffman Coding

In [15]:
frequency_map, encoding_map = huffman_coding(net, centers)
np.save("huffman_encoding", encoding_map)
np.save("huffman_freq", frequency_map)

Original storage for each parameter: 5.0000 bits
Average storage for each parameter after Huffman Coding: 2.4421 bits
Complete 1 layers for Huffman Coding...
Original storage for each parameter: 5.0000 bits
Average storage for each parameter after Huffman Coding: 2.2364 bits
Complete 2 layers for Huffman Coding...
Original storage for each parameter: 5.0000 bits
Average storage for each parameter after Huffman Coding: 2.7525 bits
Complete 3 layers for Huffman Coding...
Original storage for each parameter: 5.0000 bits
Average storage for each parameter after Huffman Coding: 2.9361 bits
Complete 4 layers for Huffman Coding...
Original storage for each parameter: 5.0000 bits
Average storage for each parameter after Huffman Coding: 2.9486 bits
Complete 5 layers for Huffman Coding...
Original storage for each parameter: 5.0000 bits
Average storage for each parameter after Huffman Coding: 2.9959 bits
Complete 6 layers for Huffman Coding...
Original storage for each parameter: 5.0000 bits
Ave

In [18]:
print(len(frequency_map[0]))
print(frequency_map)
print(encoding_map)

32
[{-0.33858323: 1, -0.28243494: 1, -0.27256134: 2, -0.25100976: 1, -0.19499578: 8, -0.17022395: 4, -0.1538356: 5, -0.14106499: 7, -0.123708375: 19, -0.10393214: 19, -0.085274965: 23, -0.07420075: 18, -0.06114363: 15, -0.045811377: 6, -0.032844998: 9, -0.009011933: 2, -1.3533281e-09: 590, 0.035615: 9, 0.053603355: 19, 0.0688159: 17, 0.085895464: 12, 0.09611583: 14, 0.109143786: 11, 0.12318396: 15, 0.13852714: 11, 0.15040794: 9, 0.1705565: 4, 0.18647346: 6, 0.21806303: 4, 0.23223442: 1, 0.28336933: 1, 0.3817434: 1}, {-0.20913272: 1, -0.17005616: 1, -0.14407216: 6, -0.13233446: 4, -0.12646312: 1, -0.114169136: 8, -0.10097553: 16, -0.09022637: 21, -0.07644305: 42, -0.06460859: 81, -0.054398473: 134, -0.044904504: 252, -0.03568582: 334, -0.02697987: 369, -0.016466243: 293, -4.2031053e-05: 6374, 0.009925877: 65, 0.016737754: 145, 0.022585075: 178, 0.028632151: 178, 0.03543756: 222, 0.043179076: 183, 0.052016772: 128, 0.06175476: 76, 0.07357724: 47, 0.08745603: 25, 0.10227048: 14, 0.1174903

### Pipeline

In [14]:
test(net)

Files already downloaded and verified
Test Loss=0.3512, Test accuracy=0.9063


In [15]:
print("-----Summary After everything-----")
summary(net)
print("-------------------------------")

-----Summary After everything-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		864			0.000000
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		9216			0.000000
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		18432			0.000000
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		36864			0.000000
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		73728			0.000000
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		147456			0.000000
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		147456			0.000000
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		294912			0.000000
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		589824			0.000000
26		BatchNorm	N/A		N/A			N/A
27		ReLU		N/A		N/A			N/A
28		Convolutional	589824		589824			0.000000
29		Bat

In [16]:
prune(net, method='std', q=45.0, s=0.75)
test(net)
print("-----Summary After everything-----")
summary(net)
print("-------------------------------")

Files already downloaded and verified
Test Loss=0.5236, Test accuracy=0.8577
-----Summary After everything-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		248			0.712963
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		2339			0.746202
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		6140			0.666884
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		12997			0.647434
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		26293			0.643378
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		49765			0.662509
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		49463			0.664558
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		86121			0.707977
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		178440			0.697469
26		BatchNorm	N/A		N/A			N/A
27	

In [17]:
finetune_after_prune(net, epochs=50, batch_size=128, lr=0.001, reg=5e-5)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=16]	Loss=0.0928	acc=0.9712	953.5 examples/second
[Step=32]	Loss=0.0925	acc=0.9700	1498.0 examples/second
[Step=48]	Loss=0.0876	acc=0.9714	1484.2 examples/second
[Step=64]	Loss=0.0890	acc=0.9705	1775.8 examples/second
[Step=80]	Loss=0.0840	acc=0.9716	2138.2 examples/second
[Step=96]	Loss=0.0840	acc=0.9714	1827.1 examples/second
[Step=112]	Loss=0.0808	acc=0.9726	1899.7 examples/second
[Step=128]	Loss=0.0791	acc=0.9733	1975.6 examples/second
[Step=144]	Loss=0.0788	acc=0.9737	1992.8 examples/second
[Step=160]	Loss=0.0777	acc=0.9741	2034.4 examples/second
[Step=176]	Loss=0.0793	acc=0.9731	2474.3 examples/second
[Step=192]	Loss=0.0817	acc=0.9726	1886.7 examples/second
[Step=208]	Loss=0.0815	acc=0.9728	1955.1 examples/second
[Step=224]	Loss=0.0810	acc=0.9730	1684.7 examples/second
[Step=240]	Loss=0.0803	acc=0.9732	2112.9 examples/second
[Step=256]	Loss=0.0805	acc=0.9732	1956.9 exam

In [4]:
# Load the best weight paramters
net.load_state_dict(torch.load("./net_after_pruning_after_finetune_i2.pt"))
test(net)
print("-----Summary After everything-----")
summary(net)
print("-------------------------------")

Files already downloaded and verified
Test Loss=0.3653, Test accuracy=0.9157
-----Summary After everything-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		248			0.712963
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		2339			0.746202
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		6140			0.666884
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		12997			0.647434
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		26293			0.643378
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		49765			0.662509
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		49463			0.664558
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		86121			0.707977
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		178440			0.697469
26		BatchNorm	N/A		N/A			N/A
27	

In [5]:
prune(net, method='std', q=45.0, s=0.75)
test(net)
print("-----Summary After everything-----")
summary(net)
print("-------------------------------")

Files already downloaded and verified
Test Loss=0.4293, Test accuracy=0.8920
-----Summary After everything-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		235			0.728009
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		2217			0.759440
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		5455			0.704047
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		11518			0.687554
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		22657			0.692695
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		42571			0.711297
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		41399			0.719245
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		72701			0.753482
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		140848			0.761203
26		BatchNorm	N/A		N/A			N/A
27	

In [6]:
finetune_after_prune(net, epochs=50, batch_size=128, lr=0.001, reg=5e-5)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=16]	Loss=0.0410	acc=0.9873	1243.9 examples/second
[Step=32]	Loss=0.0372	acc=0.9880	2442.6 examples/second
[Step=48]	Loss=0.0377	acc=0.9871	2253.8 examples/second
[Step=64]	Loss=0.0423	acc=0.9858	2848.1 examples/second
[Step=80]	Loss=0.0454	acc=0.9851	2469.8 examples/second
[Step=96]	Loss=0.0466	acc=0.9847	2502.2 examples/second
[Step=112]	Loss=0.0479	acc=0.9841	2368.2 examples/second
[Step=128]	Loss=0.0469	acc=0.9842	2456.5 examples/second
[Step=144]	Loss=0.0483	acc=0.9840	2229.9 examples/second
[Step=160]	Loss=0.0489	acc=0.9843	2160.3 examples/second
[Step=176]	Loss=0.0481	acc=0.9842	2221.1 examples/second
[Step=192]	Loss=0.0478	acc=0.9844	2242.8 examples/second
[Step=208]	Loss=0.0486	acc=0.9841	2349.3 examples/second
[Step=224]	Loss=0.0486	acc=0.9842	2321.9 examples/second
[Step=240]	Loss=0.0479	acc=0.9844	2347.4 examples/second
[Step=256]	Loss=0.0484	acc=0.9843	2677.5 exa

In [7]:
# Load the best weight paramters
net.load_state_dict(torch.load("./net_after_pruning_after_finetune_i3.pt"))
test(net)
print("-----Summary After everything-----")
summary(net)
print("-------------------------------")

Files already downloaded and verified
Test Loss=0.3838, Test accuracy=0.9148
-----Summary After everything-----
Layer id	Type		Parameter	Non-zero parameter	Sparsity(\%)
1		Convolutional	864		235			0.728009
2		BatchNorm	N/A		N/A			N/A
3		ReLU		N/A		N/A			N/A
4		Convolutional	9216		2217			0.759440
5		BatchNorm	N/A		N/A			N/A
6		ReLU		N/A		N/A			N/A
7		Convolutional	18432		5455			0.704047
8		BatchNorm	N/A		N/A			N/A
9		ReLU		N/A		N/A			N/A
10		Convolutional	36864		11518			0.687554
11		BatchNorm	N/A		N/A			N/A
12		ReLU		N/A		N/A			N/A
13		Convolutional	73728		22657			0.692695
14		BatchNorm	N/A		N/A			N/A
15		ReLU		N/A		N/A			N/A
16		Convolutional	147456		42571			0.711297
17		BatchNorm	N/A		N/A			N/A
18		ReLU		N/A		N/A			N/A
19		Convolutional	147456		41399			0.719245
20		BatchNorm	N/A		N/A			N/A
21		ReLU		N/A		N/A			N/A
22		Convolutional	294912		72701			0.753482
23		BatchNorm	N/A		N/A			N/A
24		ReLU		N/A		N/A			N/A
25		Convolutional	589824		140848			0.761203
26		BatchNorm	N/A		N/A			N/A
27	